# 뉴럴 네트워크 학습 알고리즘 구현

## Import modules

In [1]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [2]:
EPOCHS=10

## 네트워크 구조 정의

In [3]:
#tensorflow 2.0 native model
class MyModel(tf.keras.Model):
    def __init__(self): #사용할 layer 정의
        super().__init__() #상위 클래스의 생성자 호출
        self.flatten=tf.keras.layers.Flatten(input_shape=(28,28)) #-> 28*28 의 input을 vector 형태로 풀어준다.
        self.dense1=tf.keras.layers.Dense(32,activation='relu') # parameter, how much neurons --> essential 
        self.dense2=tf.keras.layers.Dense(64,activation='relu') # parameter, how much neurons --> essential 
        self.dense3=tf.keras.layers.Dense(128,activation='relu') # parameter, how much neurons --> essential 
        self.dense4=tf.keras.layers.Dense(256,activation='relu') # parameter, how much neurons --> essential 
        self.dense5=tf.keras.layers.Dense(10,activation='softmax')
        
        #각 layer에 대해 서로 연결 해준다.
        #training 여부에 따라 실행하고자 하는 action을 달리 지정할 수 있다.
    def call(self,x,training=None,mask=None):
        x=self.flatten(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        x=self.dense4(x)
        return self.dense5(x)        

## 학습 함수 구현

In [4]:
@tf.function #tensorflow function의 형태로 최적화가 진행된다.
def train_step(model,images,labels,loss_object,optimizer,train_loss,train_accuracy): #image->training, label -> output 
    with tf.GradientTape() as tape: #gradientTape 함수를 이용해서 forward operation 과정 속에서 중간결과를 저장하도록 지정한다. -> 즉 나중에 back propagation을 이용할 거니까 미리 중간결과를 저장해줘 라는 명령을 내리는 것이다.
        predictions=model(images) # 32 * 10 ->각각의 batch 개의 image에 대해서 각각의 output에 대한 확률
        loss=loss_object(labels,predictions)
    gradients=tape.gradient(loss,model.trainable_variables) #모델에 대한 모든 trainable parameter 에 대해서 미분 진행
    optimizer.apply_gradients(zip(gradients,model.trainable_variables)) #각각의 gradient에 대한 미분값에 대해서 손실 함수 최적화를 진행한다.
    
    train_loss(loss)
    train_accuracy(labels,predictions)

## 테스트 함수 구현

In [5]:
@tf.function
def test_step(model,images,labels,loss_object,train_loss,train_accuracy):
    predictions=model(images)
    loss=loss_object(labels,predictions)
    
    train_loss(loss)
    train_accuracy(labels,predictions)

## 데이터 불러오기

In [6]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train,x_test=x_train/255.0,x_test/255.0 #x_train,x_test -> 0~255 부터의 integer로 이루어진 데이터이므로 이를 float 형태로 만들어준다. normalize

train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1024).batch(32)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

11493376/11490434 [==============================] - 0s 0us/step


## 모델 생성

In [7]:
model=MyModel()

## 손실함수 및 최적화 알고리즘 정의

In [8]:
print(y_train)
print(y_train.shape)
print(type(y_train))

[5 0 4 ... 5 6 8]
(60000,)
<class 'numpy.ndarray'>


In [9]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()  
"""
label, prediction input으로 받는다. prediction -> BatchSize * classes 
label이 속하는 class만 표현하는 Sparse 형태인 경우 SparseCaterogy..함수를 사용하고, 
만약 label이 prediction과 동일한 형태로 모든 class에 대해 0,1 로 표현 되어 있는 경우 Categorical..함수를 이용한다.
"""
optimizer=tf.keras.optimizers.Adam() #Adam optimizer 활용

## 성능 지표 정의

In [10]:
train_loss=tf.keras.metrics.Mean(name='train_loss') #loss function의 값에 대해서 mean값을 구해주면 된다.
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #loss object와 같은 쓰임을 보인다. ->따라서 format을 맞춰준디.

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [11]:
for epoch in range(EPOCHS):
    for images, lables in train_ds:
        train_step(model, images, lables, loss_object, optimizer, train_loss,train_accuracy)
        
    for images, lables in train_ds:
        test_step(model, images, lables, loss_object, test_loss, test_accuracy)
        
    template='Epoch {} : Train Loss: {}, Train Accuracy : {}%, Test Loss: {}, Test Accuracy : {}%'
    #metric의 결과를 출력하기 위해 result() function을 활용한다.
    print(template.format(epoch+1,
                        train_loss.result(),train_accuracy.result()*100,
                         test_loss.result(),test_accuracy.result()*100))
    
    #성능 지표의 정의에서 mean값을 구하도록 했기 때문에 state을 초기화 하지 않으면 계속적으로 mean값을 누적해서 계산하게 된다.
    train_loss.reset_states()
    train_accuracy.reset_states()

    test_loss.reset_states()
    test_accuracy.reset_states()    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1 : Train Loss: 0.2792075574398041, Train Accuracy : 91.39666748046875%, Test Loss: 0.13998070359230042, Test Accuracy : 95.67166900634766%
Epoch 2 : Train Loss: 0.13729040324687958, Train Accuracy : 95.77333068847656%, Test Loss: 0.10610227286815643, Test Accuracy : 96.71499633789062%
Epoch 3 : Train Loss: 0.10754730552434921, Train Accuracy : 96.63666534423828%, Test Loss: 0.11202900856733322, Test Accuracy : 96.55332946777344%
Epoch 4 : Train Loss: 0.08947119116783142, Train Accuracy : 97.22833